In [4]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

In [6]:
# Путь до основного каталога
base_dir = './flowers'

# Список классов из папок
flowers = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

# Сбор путей к изображениям и меток
image_paths = []
labels = []

for flower in flowers:
    flower_dir = os.path.join(base_dir, flower)
    for fname in os.listdir(flower_dir):
        image_paths.append(os.path.join(flower_dir, fname))
        labels.append(flower) # Записываем название папки как метку класса

# Создаем DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})
df

,image_path,label
0,./flowers\daisy\100080576_f52e8ee070_n.jpg,daisy
1,./flowers\daisy\10140303196_b88d3d6cec.jpg,daisy
2,./flowers\daisy\10172379554_b296050f82_n.jpg,daisy
3,./flowers\daisy\10172567486_2748826a8b.jpg,daisy
4,./flowers\daisy\10172636503_21bededa75_n.jpg,daisy
...,...,...
4312,./flowers\tulip\9831362123_5aac525a99_n.jpg,tulip
4313,./flowers\tulip\9870557734_88eb3b9e3b_n.jpg,tulip
4314,./flowers\tulip\9947374414_fdf1d0861c_n.jpg,tulip
4315,./flowers\tulip\9947385346_3a8cacea02_n.jpg,tulip


In [8]:
# Разделяем на тренировочную и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [10]:
# Аугментация данных
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
# Создаем генераторы
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'image_path',
    y_col = 'label',
    target_size = (255, 255),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 3021 validated image filenames belonging to 5 classes.
Found 1296 validated image filenames belonging to 5 classes.


In [14]:
# Содание модели
def create_model():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(255, 255, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    return model

In [16]:
optimizers = {
    'adam': 'adam',
    'adamw': 'adamw',
    'sgd': 'sgd'
}

final_accuracies = {}

for opt_name, opt in optimizers.items():
    print(f"Обучение с оптимизатором {opt_name}.")
    model = create_model()
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_generator,
        epochs = 10,
        validation_data=test_generator,
        verbose=1
    )

    final_accuracy = history.history['val_accuracy'][-1]
    final_accuracies[opt_name] = final_accuracy
    print(f"{opt_name}: Final accuracy = {final_accuracy:.4f}")

print("\nResults")
for opt_name, accuracy in final_accuracies.items():
    print(f"{opt_name}: {accuracy:.4f}")

Обучение с оптимизатором adam.


D:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.2923 - loss: 1.6536 - val_accuracy: 0.4761 - val_loss: 1.2143
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.4647 - loss: 1.2535 - val_accuracy: 0.4946 - val_loss: 1.1384
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.5165 - loss: 1.1932 - val_accuracy: 0.5887 - val_loss: 1.0532
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.5254 - loss: 1.1250 - val_accuracy: 0.6142 - val_loss: 1.0163
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.5762 - loss: 1.0442 - val_accuracy: 0.5941 - val_loss: 1.0203
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.5823 - loss: 1.0358 - val_accuracy: 0.6566 - val_loss: 0.9044
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.6210 - loss: 0.9659 - val_accuracy: 0.6420 - val_loss: 0.9223
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.6351 - loss: 0.9456 - val_accuracy: 0.6127 - v

In [18]:
new_train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=64,
    class_mode='categorical'
)

new_test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=64,
    class_mode='categorical'
)

Found 3021 validated image filenames belonging to 5 classes.
Found 1296 validated image filenames belonging to 5 classes.


In [22]:
optimizers = {
    'adam': 'adam',
    'adamw': 'adamw',
}

final_accuracies = {}

for opt_name, opt in optimizers.items():
    print(f"Обучение с оптимизатором {opt_name}.")
    model = create_model()
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        new_train_generator,
        epochs = 15,
        validation_data=new_test_generator,
        verbose=1
    )

    final_accuracy = history.history['val_accuracy'][-1]
    final_accuracies[opt_name] = final_accuracy
    print(f"{opt_name}: Final accuracy = {final_accuracy:.4f}")

print("\nResults")
for opt_name, accuracy in final_accuracies.items():
    print(f"{opt_name}: {accuracy:.4f}")

Обучение с оптимизатором adam.


D:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2995 - loss: 1.7782

D:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.3009 - loss: 1.7724 - val_accuracy: 0.4645 - val_loss: 1.2853
Epoch 2/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.4477 - loss: 1.2446 - val_accuracy: 0.5031 - val_loss: 1.1918
Epoch 3/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.5146 - loss: 1.1331 - val_accuracy: 0.5741 - val_loss: 1.1048
Epoch 4/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5440 - loss: 1.1105 - val_accuracy: 0.5617 - val_loss: 1.0580
Epoch 5/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5536 - loss: 1.0798 - val_accuracy: 0.5934 - val_loss: 1.0185
Epoch 6/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - accuracy: 0.6091 - loss: 0.9845 - val_accuracy: 0.6489 - val_loss: 0.9246
Epoch 7/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - accuracy: 0.6072 - loss: 0.9830 - val_accuracy: 0.6505 - val_loss: 0.9339
Epoch 8/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 0.6397 - loss: 0.9556 - val_accuracy: 0.6497 - val_loss: 0.

In [24]:
model.save(f'{base_dir}\\testOptimizer.h5')